# Using EcoFOCIpy to process raw field data

## DY2206 (Spring Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [21]:
import yaml
import glob

import ecofocipy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import ecofocipy.io.ncCFsave as ncCFsave
import ecofocipy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [22]:
sample_data_dir = '/Users/bell/ecoraid/2018/CTDcasts/dy1806/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [23]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY1806' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY1806.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDoneoxy.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2018/CTDcasts/dy1806/rawconverted/ctd013.btl
Processing /

In [24]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd002.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,sbeox1ml/l,sbeox1ps,sbox1mm/kg,fleco-afl,t090c,t190c,prdm,turbwetntu0,par,scan
count,3.000000,3.00000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,7.331467,100.65400,319.705000,30.545633,31.771800,24.143700,25.114800,-1.747033,-23.985000,-76.183000,1.343367,5.018367,5.017833,18.718333,0.429767,244.441233,10513.333333
std,0.028016,0.45815,1.216896,0.036955,0.149884,0.025895,0.120184,0.005547,0.052849,0.243395,0.742135,0.032227,0.030166,16.371833,0.058157,393.628366,3292.917298
min,7.299400,100.14900,318.311000,30.523700,31.666500,24.128500,25.033100,-1.751700,-24.034000,-76.388000,0.521600,4.997700,4.998600,1.452000,0.366500,3.894700,7150.000000
25%,7.321600,100.45950,319.280000,30.524300,31.686000,24.128750,25.045800,-1.750100,-24.013000,-76.317500,1.032700,4.999800,5.000450,11.068500,0.404200,17.311850,8904.500000
50%,7.343800,100.77000,320.249000,30.524900,31.705500,24.129000,25.058500,-1.748500,-23.992000,-76.247000,1.543800,5.001900,5.002300,20.685000,0.441900,30.729000,10659.000000
75%,7.347500,100.90650,320.402000,30.556600,31.824450,24.151300,25.155650,-1.744700,-23.960500,-76.080500,1.754250,5.028700,5.027450,27.351500,0.461400,364.714500,12195.000000
max,7.351200,101.04300,320.555000,30.588300,31.943400,24.173600,25.252800,-1.740900,-23.929000,-75.914000,1.964700,5.055500,5.052600,34.018000,0.480900,698.700000,13731.000000


## Create BTL report file

In [25]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [26]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY1806',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2018, 5, 15),
 'EndDate': datetime.date(2018, 5, 30),
 'Project': 'EcoFOCI',
 'ChiefScientist': 'Steve Porter',
 'StartPort': '',
 'EndPort': '',
 'CruiseLocation': '',
 'Description': 'FOCI Spring Icthyoplankton Survey',
 'CruiseYear': 2018,
 'ctdlogs_pdf_name': ''}

In [27]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54044,
 'Vessel': 'NOAAS Oscar Dyson',
 'CruiseID': 'DY1806',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY1806',
 'Project': 'Spring Icthyoplankton Cruise',
 'StationNo_altname': '1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 54,
 'LatitudeMin': 22.158,
 'LongitudeDeg': 165,
 'LongitudeMin': 26.76,
 'GMTDay': 14,
 'GMTMonth': 'may',
 'GMTYear': 2018,
 'GMTTime': 10380,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': -9999,
 'SeaState': '-99',
 'Visibility': '-99',
 'WindDir': -999,
 'WindSpd': -99.9,
 'CloudAmt': '-99',
 'CloudType': '-99',
 'Weather': '-99',
 'SurfaceTemp': -99.9,
 'BottomDepth': -9999,
 'StationNameID': '',
 'MaxDepth': 101,
 'InstrumentSerialNos': '',
 'Notes': 'Salt taken at 90m depth, bottle #21\r',
 'NutrientBtlNiskinNo': '',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '',
 'ChlorophyllBtlVolumes': '',
 'Inst

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [28]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [29]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            # 'sbox1mm/kg':'oxy_conc_ch2',
                            # 'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            # 'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,sbeox1ml/l,sbeox1ps,sbox1mm/kg,chlor_fluorescence,temperature_ch1,temperature_ch2,pressure,turbidity,par,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,,
2.0,7.4765,104.084,325.534,32.5118,32.5118,25.6974,25.6971,-1.7219,-23.971,-74.973,3.8597,5.0507,5.0526,13.508,0.2407,25.663,16256.0,2018-05-14 03:00:01,ctd001


In [30]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'sbeox1ml/l', 'sbeox1ps',
       'sbox1mm/kg', 'chlor_fluorescence', 'temperature_ch1',
       'temperature_ch2', 'pressure', 'turbidity', 'par', 'scan', 'datetime',
       'cast'],
      dtype='object')

## Add institutional meta-information


In [31]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [32]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [33]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 3)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0
Data variables: (12/19)
    oxy_concM_ch1       (bottle) float64 6.542 7.476 7.643
    oxy_percentsat_ch1  (bottle) float64 90.45 104.1 106.9
    oxy_conc_ch1        (bottle) float64 284.8 325.5 332.8
    salinity_ch1        (bottle) float64 32.71 32.51 32.48
    salinity_ch2        (bottle) float64 32.71 32.51 32.48
    sigma_t_ch1         (bottle) float64 25.89 25.7 25.65
    ...                  ...
    pressure            (bottle) float64 91.95 13.51 2.154
    turbidity           (bottle) float64 0.1914 0.2407 0.261
    par                 (bottle) float64 0.21 25.66 208.2
    scan                (bottle) float64 7.753e+03 1.626e+04 1.9e+04
    datetime            (bottle) datetime64[ns] 2018-05-14T02:54:07 ... 2018-...
    cast                (bottle) object 'ctd001' 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [34]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [35]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=True)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [36]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 3, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/13)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 6.542 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 90.45 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 284.8 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.71 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.71 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.89 ......
    ...                  ...
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 1.13 ... ...
    temperature_ch1     (latitude, longitude, time, bottle) float64 4.706 ......
    temperature_ch2     (latitude, longitude, time, bottle) float64 4.707 ......
    pressure            (latitude, longitude, time, bottle) float64 91.95 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.1914 .....
    par                 (latitude, longitude, time, bottle) float64 0.21 ... ...

In [37]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 3, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/13)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 6.542 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 90.45 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 284.8 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.71 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.71 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.89 ......
    ...                  ...
    chlor_fluorescence  (latitude, longitude, time, bottle) float64 1.13 ... ...
    temperature_ch1     (latitude, longitude, time, bottle) float64 4.706 ......
    temperature_ch2     (latitude, longitude, time, bottle) float64 4.707 ......
    pressure            (latitude, longitude, time, bottle) float64 91.95 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.1914 .....
    par                 (latitude, longitude, time, bottle) float64 0.21 ... ...
Attributes: (12/44)
    CruiseID:                      DY1806
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2023-02-03T10:50:12Z
    date_modified:                 
    QC_indicator:                  unqcd

## Rare Bottle File Edits

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [38]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            # 'sbox1mm/kg':'oxy_conc_ch2',
                            # 'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            # 'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history